In [1]:
!git clone https://github.com/spiorf/stylegan-encoder

In [2]:
path = "../input/styleganffhq/karras2019stylegan-ffhq-1024x1024.pkl"
!ls ../input

Trying out code from this fork: https://github.com/spiorf/stylegan-encoder

In [3]:
%cd stylegan-encoder

**restart the kernel after installing tensorflow 1.14**

In [4]:

%pip install tensorflow-gpu==1.15.2
import tensorflow as tf
if tf.__version__ != '1.15.0':
    print(f"not {tf.__version__}")

tf.compat.v1.config.set_soft_device_placement(True)

In [5]:
#sfd

In [6]:
import os
import pickle
import PIL.Image
import numpy as np
import dnnlib
import dnnlib.tflib as tflib
    
import config
from encoder.generator_model import Generator

import matplotlib.pyplot as plt
%matplotlib inline


# if this cell does not execute then restart and run 
# so that tensorflow version is 1.15

In [7]:
import config
print(config.cache_dir)

In [8]:
!nvidia-smi

In [9]:
#!cat /home/.local/lib/python3.7/site-packages/keras/utils/multi_gpu_utils.py

In [10]:
!pwd

In [11]:
path = "../../input/styleganffhq/karras2019stylegan-ffhq-1024x1024.pkl"

In [12]:
!ls ../../input/styleganffhq

In [13]:
#! mv {path} ./sample.pkl

In [14]:
# URL_FFHQ = 'https://drive.google.com/uc?id=1MEGjdvVpUsu1jB4zrXZN7Y4kBBOzizDQ'
# this is the pickle file

# for this particular cell to work you need to enable your GPU

tflib.init_tf()
# with dnnlib.util.open_url(URL_FFHQ, cache_dir=config.cache_dir) as f:
#     generator_network,discriminator_network, Gs_network = pickle.load(f)

with open(path, 'rb') as pickle_file:
    generator_network, discriminator_network, Gs_network = pickle.load(pickle_file)



In [15]:
try:
    generator = Generator(Gs_network, batch_size=1, randomize_noise=False)
except Exception as e:
    print(e)
    
# we have got exception error
# lets try to chane

In [16]:
!cat /opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py | grep -n "GPU"

## these are the problematic lines

In [17]:
# replacing all instances of GPU with XLA_GPU

!sed -i "s/GPU:/XLA_GPU:/" /opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py
!cat /opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py | grep -n "GPU"


In [18]:
try:
    generator = Generator(Gs_network, batch_size=1, randomize_noise=False)
except Exception as e:
    print(e)

In [19]:
!sed '1748!d' /opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py

# I dont have any error in 1748 or mention of GPU. 

In [20]:
def generate_image(latent_vector):
    latent_vector = latent_vector.reshape((1,18,512))
    generator.set_dlatents(latent_vector)
    img_array = generator.generate_images()[0]
    img = PIL.Image.fromarray(img_array, 'RGB')
    return Image.resize((256,256))

def move_and_show(latent_vector, direction, coeffs):
    fig, ax = plt.subplots(1, len(coeffs), figsize=(15,10), dpi=80)
    for i, coeff in enumerate(coeffs):
        new_latent_vector = latent_vector.copy()
        new_latent_vector[:8] = (latent_vector + coeff*direction)[:8]
        ax[i].imshow(generate_image(new_latent_vector))
        ax[i].set_title('Coeff: %0.1f' % coeff)
    [x.axis('off') for x in ax]
    plt.show()

In [21]:
# Loading already learned representations
donald_trump = np.load('ffhq_dataset/latent_representations/donald_trump_01.npy')
hillary_clinton = np.load('ffhq_dataset/latent_representations/hillary_clinton_01.npy')

# Of course you can learn your own vectors using two scripts

# 1) Extract and align faces from images
# python align_images.py raw_images/ aligned_images/

# 2) Find latent representation of aligned images
# python encode_images.py aligned_images/ generated_images/ latent_representations/

In [22]:
# Loading already learned latent directions
smile_direction = np.load('ffhq_dataset/latent_directions/smile.npy')
gender_direction = np.load('ffhq_dataset/latent_directions/gender.npy')
age_direction = np.load('ffhq_dataset/latent_directions/age.npy')

# In general it's possible to find directions of almost any face attributes: position, hair style or color ... 
# Additional scripts for doing so will be realised soon


In [23]:
move_and_show(donald_trump, smile_direction, [-1, 0, 2])